In [1]:
import os, glob, pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
import jupyternotify

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import utils
from tensorflow.keras.callbacks import EarlyStopping, Callback

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, plot_confusion_matrix, confusion_matrix
from sklearn.preprocessing import LabelEncoder

from scripts import common_quick_plots as qp

np.random.seed(9)


In [12]:
%load_ext jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

The jupyternotify extension is already loaded. To reload it, use:
  %reload_ext jupyternotify


<IPython.core.display.Javascript object>

In [14]:
%%notify -m "Model Rendered"

import time
time.sleep(5)

<IPython.core.display.Javascript object>

### Phase 1 - Problem Definition  

#### 1.1 Broad Goals  

I have multiple small animals underfoot while I'm coding, cats and dogs.  While my dog is canine shaped and behaved, she is a small breed and ruff-ly the same size or smaller than my cat.  

#### 1.2 Data Source  

This project combines two open source datasets to create a single sample:  
    1. the [Stanford Dogs Dataset](http://vision.stanford.edu/aditya86/ImageNetDogs/) which includes a limited number of samples for each category and over 120 separate dog breeds.  
    2. the [Kaggle Cat Breeds Dataset](https://www.kaggle.com/ma7555/cat-breeds-dataset) which includes a limited number of samples for 67 different cat breeds

#### 1.3 Problem Statement 

While identifying which pet is at my feet is an easy task for me as a human, it becomes a larger challenge for a computer. This project aims to build a binary image classification model to check whether a pet is a cat or a dog from a photo, with a stretch goal of applying real-time testing to a webcam.

### Phase 2 - Data Gathering  
####    2.1 Loading Files

In [ ]:
imgs = []

def load_resize(file, size = (160, 160)):
    # load file
    try:
        img_arr = cv2.imread(file)
        #convert to color array
        img_rgb = cv2.cvtColor(img_arr, cv2.COLOR_BGR2RGB)
        #reshape to uniform size 160x160x3
        return cv2.resize(img_rgb, size)
    except:
        print(f"Invalid Path")

for path in glob.glob('./data/*'):
    species = os.path.basename(path)
    for breed in glob.glob(path +'/*'):
        # pull a simplified name
        if species == 'cat':
            breed_name = breed.split('/')[-1]
            for file in glob.glob(breed + '/*.jpg')[:350]: #limit to 350 per cat breed
                file_name = os.path.basename(file)
                # load file
                image = load_resize(file)
                imgs.append({'species' : species, 
                             'breed': breed_name, 
                             'image' : image
                            })
        else:
            breed_name = breed.split('-')[-1]
            for file in glob.glob(breed + '/*.jpg')[:150]: #limit to 150 per dog breed
                file_name = os.path.basename(file)
                # load file
                image = load_resize(file)
                imgs.append({'species' : species, 
                             'breed': breed_name, 
                             'image' : image
                            })

                
#convert list to dataframe
df = pd.DataFrame.from_dict(imgs)

In [ ]:
df.head()

In [ ]:
df.info()

### Phase 3 - Exploratory Data Analysis  

#### 3.1 Dataset Shape

In [ ]:
df['species'].value_counts()

In [ ]:
df['species'].value_counts(normalize = True)

#### 3.2 Sample Images

#### 3.3 Principal Component Analysis

### Phase 4 - Modeling  

#### 4.1 Train/Test/Split

In [4]:
X = []
y = []
for path in glob.glob('./data/*'):
    species = os.path.basename(path)
    for breed in glob.glob(path +'/*'):
        # pull a simplified name
        if species == 'cat':
            breed_name = breed.split('/')[-1]
            for file in glob.glob(breed + '/*.jpg')[:350]: #limit to 350 per cat breed
                file_name = os.path.basename(file)
                # load file
                try:
                    img_arr = cv2.imread(file)
                    #convert to color array
                    img_rgb = cv2.cvtColor(img_arr, cv2.COLOR_BGR2RGB)
                    #reshape to uniform size 160x160x3
                    new_arr =  cv2.resize(img_rgb, (160, 160))
                    X.append(new_arr)
                    y.append(1)
                except:
                    print(f"Invalid Path")
        else:
            breed_name = breed.split('-')[-1]
            for file in glob.glob(breed + '/*.jpg')[:150]: #limit to 150 per dog breed
                file_name = os.path.basename(file)
                # load file
                try:
                    img_arr = cv2.imread(file)
                    #convert to color array
                    img_rgb = cv2.cvtColor(img_arr, cv2.COLOR_BGR2RGB)
                    #reshape to uniform size 160x160x3
                    new_arr =  cv2.resize(img_rgb, (160, 160))
                    X.append(new_arr)
                    y.append(0)
                except:
                    print(f"Invalid Path")

Invalid Path
Invalid Path


In [5]:
X = np.array(X)
y = np.array(y)
X.shape, y.shape

((33102, 160, 160, 3), (33102,))

In [6]:
X = X.astype('float32')
y = utils.to_categorical(y, 2)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 9)

#### 4.2 Convolutional Neural Net

In [8]:
cnn = Sequential()
# Starting Layer
cnn.add(Conv2D(filters = 128,
               kernel_size = (3, 3),
               activation = 'relu',
               input_shape = (160, 160, 3)))
cnn.add(MaxPooling2D(pool_size = (2,2)))
#second convolutional layer
cnn.add(Conv2D(filters=64,            
                     kernel_size=(3, 3),        
                     activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2,2)))

#third convolutional layer
cnn.add(Conv2D(filters=128,            
                     kernel_size=(3, 3),        
                     activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2,2)))

#fourth convolutional layer
cnn.add(Conv2D(filters=32,            
                     kernel_size=(3, 3),        
                     activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2,2)))

#fifth convolutional layer
cnn.add(Conv2D(filters=32,            
                     kernel_size=(3, 3),        
                     activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2,2)))


#flatten the metrics to fit into the Dense layers 
cnn.add(Flatten())
cnn.add(Dense(32, activation = 'relu'))
cnn.add(Dropout(0.3))
cnn.add(Dense(256, activation = 'relu'))
cnn.add(Dropout(0.3))
cnn.add(Dense(128, activation = 'relu'))
cnn.add(Dropout(0.3))
cnn.add(Dense(64, activation = 'relu'))
cnn.add(Dropout(0.3))
cnn.add(Dense(32, activation = 'relu'))
cnn.add(Dense(2, activation = 'sigmoid'))
cnn.compile(loss = 'binary_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])
early_stop = EarlyStopping(patience = 5, restore_best_weights = True)

In [9]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 158, 158, 128)     3584      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 79, 79, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 77, 77, 64)        73792     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 38, 38, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 36, 36, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 18, 18, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 32)        3

In [10]:
%%notify -o
res = cnn.fit(X_train, y_train,
             batch_size = 64,
             validation_data = (X_test, y_test),
             epochs = 20,
             callbacks = [early_stop],
             verbose = 1)


Epoch 1/20
388/388 [==============================] - 738s 2s/step - loss: 0.6179 - accuracy: 0.6370 - val_loss: 0.3962 - val_accuracy: 0.8255
Epoch 2/20
388/388 [==============================] - 730s 2s/step - loss: 0.3138 - accuracy: 0.8728 - val_loss: 0.3437 - val_accuracy: 0.8742
Epoch 3/20
388/388 [==============================] - 719s 2s/step - loss: 0.2394 - accuracy: 0.9082 - val_loss: 0.2167 - val_accuracy: 0.9211
Epoch 4/20
388/388 [==============================] - 720s 2s/step - loss: 0.2169 - accuracy: 0.9184 - val_loss: 0.1906 - val_accuracy: 0.9300
Epoch 5/20
388/388 [==============================] - 695s 2s/step - loss: 0.2115 - accuracy: 0.9197 - val_loss: 0.2425 - val_accuracy: 0.9109
Epoch 6/20
388/388 [==============================] - 685s 2s/step - loss: 0.2060 - accuracy: 0.9237 - val_loss: 0.1868 - val_accuracy: 0.9343
Epoch 7/20
388/388 [==============================] - 1749s 5s/step - loss: 0.1859 - accuracy: 0.9316 - val_loss: 0.2207 - val_accuracy: 0.924

UsageError: Line magic function `%%notify` not found.


In [ ]:
qp.plot_loss(res, 'Loss by Epoch')

In [ ]:
qp.plot_accuracy(res, 'Accuracy by Epoch')

In [ ]:
qp.plot_confusion_matrix(cnn, X_test, y_test, "Confusion Matrix")